In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
#pip install git@github.com:badw/SMILESBOX.git
#pip install git@github.com:badw/reactit.git
#pip install mace-torch

In [ ]:
from ase.optimize import QuasiNewton
from ase.vibrations import Vibrations
from smilesbox.smilesbox import SMILESbox
from arcs.generate import GetEnergyandVibrationsAseCalc
import shutil
from mace.calculators import MACECalculator 
from mace.calculators import mace_off

For generating ARCS data using GNNs or ML Models (Compatible with ASE)


we can set up a reaction graph between the following reagents:

1. H2,
2. O2,
3. H2O
4. H2O2

In [33]:
reagents = {'H2':'[HH]','O2':'O=O','H2O':'O','H2O2':'OO'}

In [34]:
#calc = CHGNetCalculator(use_device='mps')

calc = mace_off(model='large')

dft_dict = {}
for reagent in reagents:
    print(reagent)
    try:
        shutil.rmtree('./vib/')
    except FileNotFoundError:
        pass
    sm = SMILESbox()
    sm.smiles_to_atoms(reagents[reagent])

    sm.add_box([10,10,10])

    atoms = sm.atoms
    atoms.calc = calc

    QuasiNewton(atoms).run(fmax=0.001)

    vib = Vibrations(atoms,nfree=4) # nfree = 2 or 4 but not sure which is more accurate

    vib.run()

    gevac = GetEnergyandVibrationsAseCalc(aseatomscalc=atoms,asevibrationscalc=vib)

    dft_dict[reagent] = gevac.as_dict()



Using MACE-OFF23 MODEL for MACECalculator with /Users/badw/.cache/mace/MACE-OFF23_large.model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using head Default out of ['Default']
H2
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 14:11:44      -31.719627       1.7048


/Users/badw/miniconda3/envs/py312/lib/python3.12/site-packages/mace/calculators/mace.py:143: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


BFGSLineSearch:    1[  2] 14:11:44      -31.855335       0.6992
BFGSLineSearch:    2[  4] 14:11:44      -31.867356       0.0068
BFGSLineSearch:    3[  5] 14:11:44      -31.867357       0.0005
O2
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 14:11:45    -4091.031980       0.8976
BFGSLineSearch:    1[  1] 14:11:45    -4091.136515       0.2940
BFGSLineSearch:    2[  3] 14:11:45    -4091.137748       0.0000
H2O
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 14:11:46    -2081.116613       0.6752
BFGSLineSearch:    1[  2] 14:11:47    -2081.122018       0.0583
BFGSLineSearch:    2[  3] 14:11:47    -2081.122063       0.0104
BFGSLineSearch:    3[  4] 14:11:47    -2081.122072       0.0021
BFGSLineSearch:    4[  5] 14:11:47    -2081.122072       0.0003
H2O2
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 14:11:49    -4126.507501       1.2906
BFGSLineSearch:    1[  2]

In [35]:
from reactit.reactit import ReactionGenerator

rg = ReactionGenerator(compounds=list(dft_dict))

reactions = rg.iterate()
dft_dict['reactions'] = rg.as_dict() # add to the dft_dict
reactions

100%|██████████| 11/11 [00:00<00:00, 268.48it/s]


['1 H2 + 1 H2O2 = 2 H2O',
 '2 H2 + 1 O2 = 2 H2O',
 '1 O2 + 2 H2O = 2 H2O2',
 '1 H2 + 1 O2 = 1 H2O2']

In [36]:
from arcs.generate import GraphGenerator

graph = GraphGenerator().from_dict(dft_dict=dft_dict,temperature=298.15,pressure=1.0)

/Users/badw/miniconda3/envs/py312/lib/python3.12/site-packages/ase/thermochemistry.py:823: UserWarning: 1 imag modes removed
  warn(f"{n_imag} imag modes removed", UserWarning)


sample with arcs

In [37]:
from arcs.traversal import Traversal
from arcs.generate import GenerateInitialConcentrations

gic = GenerateInitialConcentrations(graph=graph).update_ic(
    {'H2':10,'O2':10}
    )

t = Traversal(graph=graph)

data = t.sample(initial_concentrations=gic,ncpus=4,nsamples=1000)

1008it [00:03, 305.49it/s]                         


In [38]:
from arcs.analysis import AnalyseSampling
import pandas as pd 

analysis = AnalyseSampling()
stats = pd.Series(analysis.reaction_statistics(data)).sort_values(ascending=False)
stats.head(10)

2 H2 + 1 O2 = 2 H2O      1548
1 H2 + 1 O2 = 1 H2O2      652
1 O2 + 2 H2O = 2 H2O2     419
1 H2 + 1 H2O2 = 2 H2O     185
dtype: int64

In [40]:
average_data = pd.DataFrame(analysis.average_sampling(data))
average_data = average_data.loc[~(average_data==0).all(axis=1)]
average_data.sort_values(by='diff',inplace=True)
print(average_data.round(2).to_string())

      initial  mean   diff   sem   std    var
H2       10.0  0.00 -10.00  0.00  0.00   0.00
O2       10.0  3.69  -6.31  0.04  1.88   3.52
H2O2      0.0  2.62   2.62  0.07  3.75  14.08
H2O       0.0  7.38   7.38  0.07  3.75  14.08


In [43]:
pyvis_kwargs = {'width':'50%','notebook':False,"font_color":'white','directed':True}
g = analysis.result_to_pyvis(data,head=20,**pyvis_kwargs)
g.save_graph(name="example_pyvis_graph.html")

In [42]:
! open example_pyvis_graph.html 